In [ ]:
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List
import pandas as pd

model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'
img_path = " "

def compute_angle(A, B, C):
    # Your existing compute_angle function
    # ...

def calculate_pointer_angle(img):
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()
        print(keypoints_set)

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        print(keypoints_set[0])
        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]
        print(keypoints_set[0])
        print(f"A is {A[1]}")
        print(f"B is {B[1]}")
        print(f"C is {C[1]}")

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

def save_to_excel(file_name, angle_ABC, angle_BDC):
    data = {'File': [file_name], 'Angle_ABC': [angle_ABC], 'Angle_BDC': [angle_BDC]}
    df = pd.DataFrame(data)
    if not os.path.isfile('angles_results.xlsx'):
        df.to_excel('angles_results.xlsx', index=False)
    else:  # else it exists so append without writing the header
        df.to_excel('angles_results.xlsx', index=False, header=False, mode='a')

if __name__ == "__main__":
    image_output = calculate_pointer_angle(img_path)
    save_to_excel(img_path, *image_output)
